<a href="https://colab.research.google.com/github/lqcgithub/MiningofMassiveDatasets/blob/main/TextSimilarityLSH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install beautifulsoup4

In [2]:
pip install pyspark

In [3]:
pip install mmh3

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
import pyspark
from pyspark import SparkConf, SparkContext

In [6]:
import re
import numpy as np
import collections
import random
from operator import itemgetter

In [7]:
import nltk
from nltk import ngrams
import mmh3
import time

In [8]:
conf = SparkConf().setMaster("local").setAppName("Market Basket with Apriori")
sc = SparkContext.getOrCreate(conf=conf)

In [9]:
baseUrl = "https://tuoitre.vn"
url = "https://tuoitre.vn/tin-moi-nhat.htm"

In [10]:
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

titles = soup.findAll('h3', class_='title-news')
links = [link.find('a').attrs["href"] for link in titles]
data = []

for link in links:
  news = requests.get(baseUrl + link)
  soup = BeautifulSoup(news.content, "html.parser")
  if (soup.find("h1", class_="article-title")is not None):
    title = soup.find("h1", class_="article-title").text
    abstract = soup.find("h2", class_="sapo").text
    body = soup.find("div", id="main-detail-body")
    contents = body.findAll("p", recursive=False)
    contents = [content.text for content in contents]
    content = ('').join(contents)
    data.append({
        "title": title,
        "abstract": abstract,
        "content": content,
    })

In [11]:
print(data[0]["content"])

Ban hành mẫu thẻ BHYT nhỏ gọn, ép plasticQuyết định 1666/QĐ-BHXH về ban hành mẫu thẻ BHYT của Bảo hiểm xã hội Việt Nam sẽ có hiệu lực từ ngày 1-4-2021. Theo đó, mẫu thẻ BHYT mới sẽ có kích thước thẻ nhỏ gọn hơn, chỉ dài 85,6mm và rộng 53,98mm, nhỏ hơn nhiều so với trước đấy dài đến 98mm và rộng 66mm. Mã số thẻ sẽ chỉ còn 10 chữ số là mã số BHXH của người tham gia BHYT.  Mặt trước của thẻ in mã số thẻ, họ tên, ngày sinh, giới tính, mức hưởng BHYT, nơi đăng ký khám chữa bệnh ban đầu, giá trị sử dụng thẻ, nơi cấp đổi thẻ BHYT…  Mặt sau của thẻ có các thông tin về nơi cấp, đổi thẻ; hướng dẫn kiểm tra chi phí khám, chữa bệnh BHYT được hưởng… Thẻ sẽ được ép plastic sau khi in.  Đối với mẫu thẻ cũ, trong thời gian chờ đổi sang mẫu thẻ mới, nếu vẫn còn thời hạn sử dụng thì tiếp tục được dùng để khám, chữa bệnh. Phôi thẻ BHYT đã in theo mẫu cũ chưa sử dụng hết đến ngày 1-4-2021 vẫn được tiếp tục sử dụng cấp cho các người tham gia BHYT.  Bên cạnh đó, Nghị định 14/2021/NĐ-CP về xử phạt vi phạm hà

In [12]:
def preprocess(document):
    document = re.sub(r'[^\w\s]','',document)
    tokens = document.lower()
    tokens = tokens.split()
    return tokens

In [13]:
document = "Trận này, Leicester áp đảo ngay sau tiếng còi khai cuộc. Đây cũng là điều dễ hiểu, bởi đối thủ của họ - Sheffield United là đội bóng đang xếp cuối bảng và gần như cầm chắc vé xuống hạng khi kém nhóm an toàn tới 14 điểm.Phút 12, cơ hội mở tỉ số đến với Leicester sau pha dàn xếp tấn công bên cánh trái. Nhưng may mắn đã không đứng về phía đội chủ nhà khi pha đánh đầu về góc xa của Ayoze Perez lại đưa bóng trúng cột dọc khung thành.  Những phút sau đó, Leicester tiếp tục tấn công và tạo ra thêm nhiều tình huống nguy hiểm nữa. Dù áp đảo hoàn toàn, nhưng phải đến cuối hiệp 1 các học trò HLV Brendan Rodgers mới đưa được bóng vào lưới khung thành thủ môn Aaron Ramsdale.  Phút 39, từ pha lên bóng bên cánh trái, Jamie Vardy bứt tốc xâm nhập vòng cấm đón đường chọc khe của đồng đội rồi thực hiện đường căng ngang vào giữa. Trong tư thế không người kèm, Kelechi Iheanacho dễ dàng ập vào đệm bóng tung lưới thủ môn Aaron Ramsdale từ cự li gần, mở tỉ số cho Leicester.  Bị thủng lưới, Sheffield United đẩy cao đội hình tấn công trong hiệp 2 và cơ hội tiếp tục đến với Leicester khi hàng phòng ngự đội khách để lộ ra nhiều khoảng trống.  Phút 64, sau vài cơ hội bị bỏ lỡ, đội chủ nhà cũng đã có bàn thắng nhân đôi cách biệt nhờ công tiền vệ Ayoze Perez với pha dứt điểm hiểm hóc ngay mép vòng cấm sau tình huống phản công bên cánh phải.  Sau bàn thua này, các cầu thủ Sheffield United càng thi đấu thiếu tập trung, đặc biệt là trong phòng ngự.  Hệ quả, chỉ trong 11 phút từ 69 đến 80, lưới của thủ môn Aaron Ramsdale đã phải rung lên thêm 3 lần nữa sau những cú dứt điểm trong tư thế vô cùng thoải mái của Kelechi Iheanacho (phút 69 và 78) và Jamie Vardy (phút 80), giúp Leicester khép lại trận đấu với chiến thắng cách biệt 5-0.  Thắng trận này, Leicester đã tạm vượt qua Manchester United (chưa đá vòng 28) để vươn lên vị trí thứ hai trên bảng xếp hạng với 56 điểm, hơn Man Utd 2 điểm và kém đội đầu bảng Man City 15 điểm. "
preprocess(document)[0:5]

['trận', 'này', 'leicester', 'áp', 'đảo']

In [14]:
doc_as_shingles = []
for document in data:
  tokens = preprocess(document['content'])
  shingles = ngrams(tokens, 3)
  doc_as_shingles.append(list(shingles))
print(doc_as_shingles[0])

[('ban', 'hành', 'mẫu'), ('hành', 'mẫu', 'thẻ'), ('mẫu', 'thẻ', 'bhyt'), ('thẻ', 'bhyt', 'nhỏ'), ('bhyt', 'nhỏ', 'gọn'), ('nhỏ', 'gọn', 'ép'), ('gọn', 'ép', 'plasticquyết'), ('ép', 'plasticquyết', 'định'), ('plasticquyết', 'định', '1666qđbhxh'), ('định', '1666qđbhxh', 'về'), ('1666qđbhxh', 'về', 'ban'), ('về', 'ban', 'hành'), ('ban', 'hành', 'mẫu'), ('hành', 'mẫu', 'thẻ'), ('mẫu', 'thẻ', 'bhyt'), ('thẻ', 'bhyt', 'của'), ('bhyt', 'của', 'bảo'), ('của', 'bảo', 'hiểm'), ('bảo', 'hiểm', 'xã'), ('hiểm', 'xã', 'hội'), ('xã', 'hội', 'việt'), ('hội', 'việt', 'nam'), ('việt', 'nam', 'sẽ'), ('nam', 'sẽ', 'có'), ('sẽ', 'có', 'hiệu'), ('có', 'hiệu', 'lực'), ('hiệu', 'lực', 'từ'), ('lực', 'từ', 'ngày'), ('từ', 'ngày', '142021'), ('ngày', '142021', 'theo'), ('142021', 'theo', 'đó'), ('theo', 'đó', 'mẫu'), ('đó', 'mẫu', 'thẻ'), ('mẫu', 'thẻ', 'bhyt'), ('thẻ', 'bhyt', 'mới'), ('bhyt', 'mới', 'sẽ'), ('mới', 'sẽ', 'có'), ('sẽ', 'có', 'kích'), ('có', 'kích', 'thước'), ('kích', 'thước', 'thẻ'), ('thước', 

In [15]:
doc_hashed = []

def hash_doc(shingles):
  shingles_hashed = []
  for shingle in shingles:
    hash = mmh3.hash(' '.join(shingle))
    shingles_hashed.append(hash)
  return shingles_hashed  

for document in doc_as_shingles:
  doc_hashed.append(list(hash_doc(document)));
print(doc_hashed[0])

[-1314682866, 1512495620, 1578336557, -384030479, -2138173164, -1168925846, 425156207, 1569914609, -899319234, -1269462426, -805677406, -397072196, -1314682866, 1512495620, 1578336557, 1585270207, 2015479985, 472767293, 1272563501, 483912037, -849105124, 1002279553, -657020686, -2003373570, 1525635398, -2030590563, 867337155, -2006360871, -523701685, 1910167492, 1362817987, 684637163, 305526748, 1578336557, -2012215997, -295843005, 452866670, 1860812968, -1891276353, 201845201, 1617150284, 1277729913, 2029304753, 23716078, -1210645992, -1292589500, -1869315016, 2033190856, -1953391756, 242945401, -64155033, 490632146, 1614301564, -1317676369, 889842505, -427800478, 484521937, -1747687117, 1898139630, -136465933, 1487578398, -765437064, -2140218179, -1393809125, 1323855485, -905453402, -877346617, 1052107940, -322762229, -441700280, -311405049, -1797002169, 1605544145, -2051495460, 741922168, 1990923481, -715391983, -1294328713, 356987367, -1912109305, -122311478, -932875868, -344316020

In [16]:
start = time.time()
signatures = []
max_shingle_id = 2 ** 32 - 1

next_prime = 4294967311

def pick_random_coeffs(k):
    rand_list = []

    while k > 0:
        rand_index = random.randint(0, max_shingle_id)

        while rand_index in rand_list:
            rand_index = random.randint(0, max_shingle_id)

        rand_list.append(rand_index)
        k = k - 1
    return rand_list

coeff_a = pick_random_coeffs(10)
coeff_b = pick_random_coeffs(10)

for document in doc_hashed:
  signature =[]
  for i in range(0, 10):
    min_hash_code = next_prime + 1
    for shingle in document: 
      hash_code = (coeff_a[i] * shingle + coeff_b[i]) % next_prime
      if hash_code < min_hash_code:
        min_hash_code = hash_code
    signature.append(min_hash_code)
  signatures.append(signature)
end = time.time()
print(signatures[0])
print(len(signatures),len(signatures[0]))
print("Minhash completed in : {:.4f} sec".format(end - start))

[3800170, 9964899, 4306758, 6029996, 290362, 19139444, 2422899, 3704649, 7208066, 10448618]
20 10
Minhash completed in : 0.0933 sec


In [17]:
def jaccardSim(d1,d2):
    return len(d1.intersection(d2))/len(d1.union(d2))   

In [18]:
sims = jaccardSim(set(signatures[0]), set(signatures[1])) 
sims

0.0

In [19]:
def initialize_array_bucket(bands):
    len_buckets = 101
    array_buckets = []
    for band in range(bands):
        array_buckets.append([[] for i in range(len_buckets)])
    return array_buckets

In [24]:
def apply_LSH_technique(signatures,t,bands,rows):
    if bands * rows != len(signatures):
        print('split to bands error.')

    array_buckets = initialize_array_bucket(bands)

    # hash_funcs = pick_random_coeffs(10);


    candidates = {}
    
    i = 0
    for b in range(bands):
        buckets = array_buckets[b]        
        band = np.array(signatures)[i:i+rows,:]
        for col in range(band.shape[1]):    
            
            key = int(sum(band[:,col]) % len(buckets))
            
            buckets[key].append(col)
        i = i+rows

        for item in buckets:
            if len(item) > 1:
                pair = (item[0], item[1])
                if pair not in candidates:
                    A = np.array(signatures)[:,item[0]]
                    B = np.array(signatures)[:,item[1]]
                    similarity = cosine_distance(A,B)
                    if similarity >= t:
                        candidates[pair] = similarity


    sort = sorted(candidates.items(),key=itemgetter(1), reverse=True)

    return candidates,sort

def euclidean_distance(x,y,r=2.0):
    try:
        
        return sum(((x[i] - y[i]) ** r) for i in range(len(x))) ** (1.0/r)
    
    except (ValueError,ZeroDivisionError):
        print ('Please, enter only even values for "r > 0".')
    except IndexError:
        print ('Please, the sets must have the same size.')

def cosine_distance(x,y):
    prodAB = sum([x[i]*y[i] for i in range(len(x))])
    zeros = [0 for i in range(len(x))]
    A = euclidean_distance(x,zeros)
    B = euclidean_distance(y,zeros)
    return prodAB / (A*B)

start = time.time()
candidates, sort = apply_LSH_technique(signatures, 0.3, 4, 5)
end = time.time()
print(candidates)
print("LSH completed in: {:.4f}".format(end - start))

{(4, 7): 0.33585297927870256}
LSH completed in: 0.0012


In [21]:
# def crawNewsData(baseUrl, url):
#     response = requests.get(url)
#     soup = BeautifulSoup(response.content, "html.parser")

#     titles = soup.findAll('h3', class_='title-news')
#     links = [link.find('a').attrs["href"] for link in titles]
#     data = []
#     for link in links:
#         news = requests.get(baseUrl + link)
#         soup = BeautifulSoup(news.content, "html.parser")
#         title = soup.find("h1", class_="article-title").text
#         abstract = soup.find("h2", class_="sapo").text
#         body = soup.find("div", id="main-detail-body")
#         content = ""
#         try:
#             contents = body.findAll("p", recursive=False)
#             contents = [content.text for content in contents]
#             content = ('').join(contents)
#         except:
#             content = ""
#         data.append({
#             "title": title,
#             "abstract": abstract,
#             "content": content,
#         })
#     return data